In [149]:
import pandas as pd
import time
import requests, time
from bs4 import BeautifulSoup
import bs4.element
from fake_useragent import UserAgent
import re

requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = "ALL:@SECLEVEL=1"

2772104168722000005 1700200833623000023 1380805025523000091 1380805025523000091 3184110207323000038 3633001779723000034 1380805025523000091

In [150]:
url_info = "https://zakupki.gov.ru/epz/contract/contractCard/common-info.html?reestrNumber="
url_payment = "https://zakupki.gov.ru/epz/contract/contractCard/payment-info-and-target-of-order.html?reestrNumber="
url_process = "https://zakupki.gov.ru/epz/contract/contractCard/process-info.html?reestrNumber="
url_version = (
    "https://zakupki.gov.ru/epz/contract/contractCard/journal-version.html?reestrNumber="
)
url_document = "https://zakupki.gov.ru/epz/contract/contractCard/document-info.html?reestrNumber="
url_event = "https://zakupki.gov.ru/epz/contract/contractCard/event-journal.html?reestrNumber="

## Получение информации с url_info

In [151]:
number_contract = "0307300000714000007"
url_info = (
    "https://zakupki.gov.ru/epz/contract/contractCard/common-info.html?reestrNumber="
    + number_contract
)
res = requests.get(url_info, headers={"User-Agent": UserAgent().random})
soup = BeautifulSoup(res.text, "html")
res.ok

True

In [152]:
soup.find("a", class_="phones-links__item")
soup.find("div", class_="hotline-phones")

In [153]:
soup.find("a", class_="phones-links__item")
soup.find("div", class_="hotline-phones")

In [154]:
soup

<!DOCTYPE html>
<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="app-id=1457694118" name="apple-itunes-app"/>
<meta content="app-id=ru.gov.zakupki.mobile" name="google-play-app"/>
<title>Карточка контракта</title>
<script type="text/javascript">
      var contextPath = "/epz/contract";
  </script>
<script src="/static/useractivityrecordplugin/js/recordSupportSystem.js" type="text/javascript"></script>
<link href="/epz/static/css/app.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/epz/static/css/jquery_msgbox.css" rel="stylesheet" type="text/css"/>
<script src="/epz/static/js/jquery-3.3.1.min.js" type="text/javascript"></script>
<script src="/epz/static/js/jquery.cookie.min.js" type="text/javascript"></script>
<script src="/epz/static/js/js.cookie.js" type="text/javascript"></script>
<script src="/epz/static/js/jquery_msgBox.js" type="text/javascript"></script>
<script type="text/javascript">
    
    $(window).ready(func

In [155]:
def remove_bad_symbols(string: str) -> str:
    string = re.sub("\n|\||\xa0", "", string.strip())
    return " ".join(string.split())

## Информация о заказчике

In [156]:
# ПОЛНОЕ НАИМЕНОВАНИЕ ЗАКАЗЧИКА
def find_full_name_customer(soup: BeautifulSoup) -> str:
    try:
        soup = soup.find("span", string="Полное наименование заказчика")
        castomer = soup.parent.find("a", target="_blank")
        castomer = castomer.get_text()
        return remove_bad_symbols(castomer)
    except AttributeError:
        # логии
        return None


find_full_name_customer(soup)

'ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИЯ РЕСПУБЛИКИ КОМИ "СЫКТЫВКАРСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 2"'

In [157]:
link = soup.find("span", class_="cardMainInfo__content").find("a")["href"]
number = re.search("Id=[0-9]+", link)
number
# re.search("[0-9]+", number)

In [158]:
# УНИКАЛЬНЫЙ УЧЕТНЫЙ НОМЕР ОРГАНИЗАЦИИ
def find_unique_account_number(soup: BeautifulSoup) -> str:
    try:
        link = soup.find("span", class_="cardMainInfo__content").find("a")["href"]
        number = re.search("Code=[0-9]+", link)
        if number is None:
            number = re.search("Id=[0-9]+", link)
            if number is None:
                return None, None
            else:
                return None, re.search("[0-9]+", number[0])[0]
        else:
            return re.search("[0-9]+", number[0])[0], None
    except AttributeError:
        # логии
        return None


find_unique_account_number(soup)

('03073000007', None)

In [159]:
# СОКРАЩЕННОЕ НАИМЕНОВАНИЕ ЗАКАЗЧИКА
def find_short_name_customer(soup: BeautifulSoup) -> str:
    try:
        soup = soup.find("span", string="Сокращенное наименование заказчика")
        castomer = soup.parent.find("span", class_="section__info")
        castomer = castomer.get_text()
        return remove_bad_symbols(castomer)
    except AttributeError:
        # логии
        return None


find_short_name_customer(soup)

'ГБУЗ РК "СДП №2"'

In [160]:
# ИДЕНТИФИКАЦИОННЫЙ КОД ЗАКАЗЧИКА
def find_id_customer(soup: BeautifulSoup) -> str:
    try:
        soup = soup.find("span", string="Идентификационный код заказчика")
        code = soup.parent.find("span", class_="section__info")
        code = code.get_text()
        return remove_bad_symbols(code)
    except AttributeError:
        # логии
        return None


find_id_customer(soup)

'21101487375110101001'

In [161]:
# ИНН ЗАКАЗЧИКА
def find_inn_customer(soup: object) -> str:
    try:
        soup = soup.find("span", string="ИНН")
        code = soup.parent.find("span", class_="section__info")
        code = code.get_text()
        return remove_bad_symbols(code)
    except AttributeError:
        # логии
        return None


find_inn_customer(soup)

'1101487375'

In [162]:
# КПП ЗАКАЗЧИКА
def find_kpp_customer(soup: object) -> str:
    try:
        soup = soup.find("span", string="КПП")
        code = soup.parent.find("span", class_="section__info")
        code = code.get_text()
        return remove_bad_symbols(code)
    except AttributeError:
        # логии
        return None


find_kpp_customer(soup)

'110101001'

In [163]:
# КОД ОРАГИЗАЦИОННО-ПРАВОВОЙ ФОРМЫ
def find_code_form_org(soup: object) -> str:
    try:
        soup = soup.find("span", string="Код организационно-правовой формы")
        code = soup.parent.find("span", class_="section__info")
        code = code.get_text()
        return remove_bad_symbols(code)
    except AttributeError:
        # логии
        return None


find_code_form_org(soup)

'75203'

In [164]:
# КОД ПО ОКПО
def find_okpo_code(soup: object) -> str:
    try:
        soup = soup.find("span", string="Код ОКПО")
        code = soup.parent.find("span", class_="section__info")
        code = code.get_text()
        return remove_bad_symbols(code)
    except AttributeError:
        # логии
        return None


find_okpo_code(soup)

'24959799'

In [165]:
# КОД ТЕРРИТОРИИ МУНИЦИПАЛЬНОГО ОБРАЗОВАНИЯ
def find_municipal_code(soup: object) -> str:
    try:
        soup = soup.find("span", string="Код территории муниципального образования")
        code = soup.parent.find("span", class_="section__info")
        code = code.get_text()
        return remove_bad_symbols(code)
    except AttributeError:
        # логии
        return None


find_municipal_code(soup)

'87701000 – Сыктывкар'

In [166]:
def find_budget_name(soup: object) -> str:
    try:
        soup_1 = soup.find("span", string="Наименование бюджета")
        name = soup_1.parent.find("span", class_="section__info")
        name = name.get_text()
        return remove_bad_symbols(name), None
    except AttributeError:
        try:
            soup = soup.find("span", string="Наименование внебюджетных средств")
            name = soup.parent.find("span", class_="section__info")
            name = name.get_text()
            return None, remove_bad_symbols(name)
        except AttributeError:
            # логии
            return None, None


find_budget_name(soup)

('республиканский бюджет Республики Коми', None)

In [167]:
def find_budget_level(soup: object) -> str:
    try:
        soup = soup.find("span", string="Уровень бюджета")
        code = soup.parent.find("span", class_="section__info")
        code = code.get_text()
        return remove_bad_symbols(code)
    except AttributeError:
        # логии
        return None


find_budget_level(soup)

'12-бюджет субъекта Российской Федерации'

## Общая информация

In [168]:
# СТАТУС КОНТРАКТА
def find_contract_status(soup: object) -> str:
    try:
        soup = soup.find("span", string="Статус контракта")
        code = soup.parent.find("span", class_="section__info")
        code = code.get_text()
        return remove_bad_symbols(code)
    except AttributeError:
        # логии
        return None


find_contract_status(soup)

'Исполнение завершено'

In [169]:
# НОМЕР ИЗВЕЩЕНИЯ ОБ ОСУЩЕСТВЛЕНИИ ЗАКУПКИ
def find_notice(soup: object) -> str:
    try:
        soup = soup.find("span", string="Номер извещения об осуществлении закупки")
        code = soup.parent.find("span", class_="section__info")
        code = code.get_text()
        return remove_bad_symbols(code)
    except AttributeError:
        # логии
        return None


find_notice(soup)

'0307300000714000003'

In [170]:
# ИДЕНТИФИКАЦИОННЫЙ КОД ЗАКУПКИ (ИКЗ)
def find_ikz_code(soup: object) -> str:
    try:
        soup = soup.find("span", string="Идентификационный код закупки (ИКЗ)")
        code = soup.parent.find("span", class_="section__info")
        code = code.get_text()
        return remove_bad_symbols(code)
    except AttributeError:
        # логии
        return None


find_ikz_code(soup)

In [171]:
# ИДЕНТИФИКАТОР КОНТРАКТА, ЗАКЛЮЧЕННОГО В ЭЛЕКТРОННОЙ ФОРМЕ
def find_id_contract_electronic(soup: object) -> str:
    try:
        soup = soup.find(
            "span", string="Идентификатор контракта, заключенного в электронной форме"
        )
        code = soup.parent.find("span", class_="section__info")
        code = code.get_text()
        return remove_bad_symbols(code)
    except AttributeError:
        # логии
        return None


find_id_contract_electronic(soup)

In [172]:
# УНИКАЛЬНЫЙ НОМЕР ПОЗИЦИИ ПЛАНА-ГРАФИКА
def find_unique_number_plan(soup: object) -> str:
    try:
        soup = soup.find("span", string="Уникальный номер позиции плана-графика")
        number = soup.parent.find("span", class_="section__info")
        number = number.get_text()
        return remove_bad_symbols(number)
    except AttributeError:
        # логии
        return None


find_unique_number_plan(soup)

In [173]:
# СПОСОБ ОПРЕДЕЛНИЯ ПОСТАВЩИКА (подрядчика, исполнителя)
def find_method_determinig_supplier(soup: object) -> str:
    try:
        soup = soup.find("span", string="Способ определения поставщика (подрядчика, исполнителя)")
        number = soup.parent.find("span", class_="section__info")
        number = number.get_text()
        return remove_bad_symbols(number)
    except AttributeError:
        # логии
        return None


find_method_determinig_supplier(soup)

'Закупка у единственного поставщика (подрядчика, исполнителя)'

In [174]:
# ДАТА ПОДВЕДЕНИЯ РЕЗУЛЬТАТОВ ОПРЕДЕЛЕНИЯ ПОСТАВЩИКА (ПОДРЯДЧИКА, ИСПОЛНИТЕЛЯ)
def find_date_summarizing(soup: object) -> str:
    try:
        soup = soup.find(
            "span",
            string="Дата подведения результатов определения поставщика (подрядчика, исполнителя)",
        )
        date = soup.parent.find("span", class_="section__info")
        date = date.get_text()
        return remove_bad_symbols(date)
    except AttributeError:
        # логии
        return None


find_date_summarizing(soup)

In [175]:
# ДАТА РАЗМЕЩЕНИЯ (ПО МЕСТНОМУ ВРЕМЕНИ)
def find_date_posting(soup: object) -> str:
    try:
        soup = soup.find("span", string="Дата размещения (по местному времени)")
        date = soup.parent.find("span", class_="section__info")
        date = date.get_text()
        return remove_bad_symbols(date)
    except AttributeError:
        # логии
        return None


find_date_posting(soup)

'28.01.2015 14:13 (МСК)'

In [176]:
# ОСНОВАНИЕ ЗАКЛЮЧЕНИЯ КОНТРАКТА С ЕДИНСТВЕННЫМ ПОСТАВЩИКОМ
def find_grouds_single_supplier(soup: object) -> str:
    try:
        soup = soup.find(
            "span", string="Основание заключения контракта с единственным поставщиком"
        )
        print()
        grouds = soup.parent.find("span", class_="section__info")
        grouds = grouds.get_text()
        return remove_bad_symbols(grouds)
    except AttributeError:
        # логии
        return None


find_grouds_single_supplier(soup)

'Часть 1 пункт 29 статьи 93 – заключение договора энергоснабжения или договора купли-продажи электрической энергии с гарантирующим поставщиком электрической энергии'

In [177]:
# РЕКВИЗИТЫ ДОКУМЕНТА, ПОДТВЕРЖДАЮЩЕГО ЗАКЛЮЧЕНИЯ КОНТРАКТА
def find_document_details(soup: object) -> str:
    try:
        soup = soup.find(
            "span", string="Реквизиты документа, подтверждающего основание заключения контракта"
        )
        date = soup.parent.find("span", class_="section__info")
        date = date.get_text()
        return remove_bad_symbols(date)
    except AttributeError:
        # логии
        return None


find_document_details(soup)

In [178]:
# ИНФОРМАЦИЯ О БАНКОВСКОМ и (или) КАЗНАЧЕЙСКОМ СОПРОВОЖДЕНИИ КОНТРАКТА
def info_support(soup: object) -> str:
    try:
        soup = soup.find(
            "span", string="Информация о банковском и (или) казначейском сопровождении контракта"
        )
        date = soup.parent.find("span", class_="section__info")
        date = date.get_text()
        return remove_bad_symbols(date)
    except AttributeError:
        # логии
        return None


info_support(soup)

'Банковское или казначейское сопровождение контракта не требуется'

## Общие данные

In [179]:
# ЗАКЛЮЧЕНИЕ КОНТРАКТА
def find_date_contract(soup: object) -> str:
    try:
        soup = soup.find("span", string="Заключение контракта")
        number = soup.parent.find("span", class_="cardMainInfo__content")
        number = number.get_text()
        return remove_bad_symbols(number)
    except AttributeError:
        # логии
        return None


find_date_contract(soup)

'07.04.2014'

In [180]:
# СРОК ИСПОЛНЕНИЯ
def find_date_performance(soup: object) -> str:
    try:
        soup = soup.find("span", string="Срок исполнения")
        number = soup.parent.find("span", class_="cardMainInfo__content")
        number = number.get_text()
        return remove_bad_symbols(number)
    except AttributeError:
        # логии
        return None


find_date_performance(soup)

'31.01.2015'

In [181]:
# ДАТА РАЗМЕЩЕНИЕ КОНТРАКТА В РЕЕСТРЕ КОНТРАКТОВ
def find_date_contract_registry(soup: object) -> str:
    try:
        soup = soup.find("span", string="Размещен контракт в реестре контрактов")
        number = soup.parent.find("span", class_="cardMainInfo__content")
        number = number.get_text()
        return remove_bad_symbols(number)
    except AttributeError:
        # логии
        return None


find_date_contract_registry(soup)

'07.04.2014'

In [182]:
# ДАТА ОБНОВЛЕНИЯ КОНТРАКТА В РЕЕСТРЕ КОНТРАКТОВ
def find_date_update_registry(soup: object) -> str:
    try:
        soup = soup.find("span", string="Обновлен контракт в реестре контрактов")
        number = soup.parent.find("span", class_="cardMainInfo__content")
        number = number.get_text()
        return remove_bad_symbols(number)
    except AttributeError:
        # логии
        return None


find_date_contract_registry(soup)

'07.04.2014'

In [183]:
number_contract = "3690202684015000006"
number_another = '1027800704818000089'
url_info = (
    "https://zakupki.gov.ru/epz/contract/contractCard/common-info.html?reestrNumber="
    + number_contract
)
res = requests.get(url_info, headers={"User-Agent": UserAgent().random})
soup = BeautifulSoup(res.text, "html")
res.ok

True

In [184]:
# ДАТА НАЧАЛА ИСПОЛНЕНИЯ КОНТРАКТА
def find_date_start_performance(soup: object) -> str:
    try:
        soup_new = soup.find("span", string="Дата начала исполнения контракта")
        date = soup_new.parent.find("span", class_="section__info")
        date = date.get_text()
        date = remove_bad_symbols(date)
        if date == 'Дата начала исполнения контракта':
            soup = soup.find("span", string="Дата начала исполнения контракта", class_='section__title')
            date = soup.parent.find("span", class_="section__info")
            date = date.get_text()
            date = remove_bad_symbols(date)
        return date
    except AttributeError:
        # логии
        return None

find_date_start_performance(soup)

'12.01.2015 Загрузка ...'

In [185]:
# ДАТА ОКОНЧАНИЯ ИСПОЛНЕНИЯ КОНТРАКТА
def find_date_end_performance(soup: object) -> str:
    try:
        soup_new = soup.find("span", string="Дата окончания исполнения контракта")
        date = soup_new.parent.find("span", class_="section__info")
        date = date.get_text()
        date = remove_bad_symbols(date)
        if date == 'Дата окончания исполнения контракта':
            soup = soup.find("span", string="Дата окончания исполнения контракта", class_='section__title')
            date = soup.parent.find("span", class_="section__info")
            date = date.get_text()
            date = remove_bad_symbols(date)
        return date
    except AttributeError:
        # логии
        return None

find_date_end_performance(soup)

'30.06.2015'

In [186]:
# ПРЕДМЕТ КОНТРАКТА
def find_contract_item(soup: object) -> str:
    try:
        soup = soup.find("span", string="Предмет контракта")
        item = soup.parent.find("span", class_="section__info")
        item = item.get_text()
        return remove_bad_symbols(item)
    except AttributeError:
        # логии
        return None


find_contract_item(soup)

In [187]:
# ЦЕНА КОНТРАКТА
def find_contract_price(soup: object) -> str:
    try:
        soup_firts = soup.find("span", string="Цена контракта")
        price = soup_firts.parent.find("span", class_="cardMainInfo__content cost")
        price = remove_bad_symbols(price.get_text())
        price = "".join([i for i in price if i.isdigit() or i in [",", "."]]).replace(",", ".")
        if price != "":
            return price
    except AttributeError:
        # логии
        pass

    try:
        soup = soup.find("div", class_="price")
        price = soup.parent.find("span", class_="cardMainInfo__content cost")
        price = remove_bad_symbols(price.get_text())
        price = "".join([i for i in price if i.isdigit() or i in [",", "."]]).replace(",", ".")
        if price != "":
            return price
    except AttributeError:
        # логии
        return None


find_contract_price(soup)

'21701.08'

In [188]:
# В ТОМ ЧИСЛЕ С НДС
def find_contract_price_nds(soup: object) -> str:
    try:
        soup_firts = soup.find("span", string="В том числе НДС")
        price = soup_firts.parent.find("span", class_="section__info")
        price = remove_bad_symbols(price.get_text())
        price = "".join([i for i in price if i.isdigit() or i in [",", "."]]).replace(",", ".")
        if price != "":
            return price
    except AttributeError:
        # логии
        return None


find_contract_price(soup)

'21701.08'

In [189]:
# РАЗМЕР АНВАНСА
def find_prepayment_amount(soup: object) -> str:
    try:
        soup = soup.find("span", string="Размер аванса")
        prepayment = soup.parent.find("span", class_="section__info")
        prepayment = remove_bad_symbols(prepayment.get_text())
        return prepayment
    except AttributeError:
        # логги
        return None


find_prepayment_amount(soup)

In [190]:
# РАЗМЕР ОБЕСПЕЧЕНИЯ ИСПОЛНЕНИЯ КОНТРАКТА
def find_performance_security(soup: object) -> str:
    try:
        soup = soup.find("span", string="Размер обеспечения исполнения контракта, ₽")
        security_sum = soup.parent.find("span", class_="section__info")
        security_sum = remove_bad_symbols(security_sum.get_text())
        security_sum = "".join(
            [i for i in security_sum if i.isdigit() or i in [",", "."]]
        ).replace(",", ".")
        if security_sum != "":
            return security_sum
    except AttributeError:
        # логии
        return None


find_performance_security(soup)

In [191]:
# РАЗМЕР ОБЕСПЕЧЕНИЯ ИСПОЛНЕНИЯ ОБЯЗАТЕЛЬСТВ ПО ПРЕДОСТАВЛЕННОЙ ГАРАНТИИ КАЧЕСТВА ТОВАРОВ, РАБОТ, УСЛУГ
def find_size_performance_quality(soup: object) -> str:
    try:
        soup = soup.find(
            "span",
            string="Размер обеспечения исполнения обязательств по предоставленной гарантии качества товаров, работ, услуг",
        )
        security_sum = soup.parent.find("span", class_="section__info")
        security_sum = remove_bad_symbols(security_sum.get_text())
        security_sum = "".join(
            [i for i in security_sum if i.isdigit() or i in [",", "."]]
        ).replace(",", ".")
        if security_sum != "":
            return security_sum
    except AttributeError:
        # логии
        return None


find_size_performance_quality(soup)

In [192]:
# СРОК НА КОТОРЫЙ ПРЕДОСТАВЛЯЕТСЯ ГАРАНТИЯ
def find_warranty_period(soup: object) -> str:
    try:
        soup_new = soup.find("h2", string="Срок, на который предоставляется гарантия")
        if soup_new is None:
            soup_new = soup.find("span", string="Срок, на который предоставляется гарантия")
        date = soup_new.parent.find("span", class_="section__info")
        date = date.get_text()
        return remove_bad_symbols(date)
    except AttributeError:
        return None


find_warranty_period(soup)

In [193]:
# ИНФОРМАЦИЯ О МЕСТЕ ПОСТАВКИ ТОВАРА, ВЫПОЛНЕНИИ РАБОТЫ ИЛИ ОКАЗАНИИ УСЛУГИ
def find_place_performance(soup: object) -> str:
    try:
        place = soup.find(
            "h2",
            string="Информация о месте поставки товара, выполнения работы или оказания услуги",
            class_="blockInfo__title_sub",
        ).parent.find("span", class_="section__info")
        place = remove_bad_symbols(place.get_text())
        return place
    except AttributeError:
        # логии
        return None


find_place_performance(soup)

## Информация о поставщиках

In [194]:
# ИМЯ ПОСТАВЩИКА
def find_full_name_supplier(soup: object) -> str:
    try:
        name = soup.next.get_text()
        return remove_bad_symbols(name)
    except AttributeError:
        # логии
        return None

In [195]:
# КОД ПО ОКПО
def find_code_okpo_supplier(soup: object) -> str:
    try:
        soup = soup.find("span", string="Код по ОКПО:")
        code = soup.next.next.next
        code = code.get_text()
        return remove_bad_symbols(code)
    except AttributeError:
        # логии
        return None

In [196]:
# ИНН ПОСТАВЩИКА
def find_inn_supplier(soup: object) -> str:
    try:
        inn = soup.parent.find("span", string="ИНН:").next.next.next
        inn = inn.get_text()
        return remove_bad_symbols(inn)
    except AttributeError:
        # логии
        return None

In [197]:
# КПП ПОСТАВЩИКА
def find_kpp_supplier(soup: object) -> str:
    try:
        kpp = soup.parent.find("span", string="КПП:").next.next.next
        kpp = kpp.get_text()
        return remove_bad_symbols(kpp)
    except AttributeError:
        # логии
        return None

In [198]:
# ДАТА ПОСТАНОВКИ НА УЧЕТ
def find_date_registration_supplier(soup: object) -> str:
    try:
        date = soup.parent.find("span", string="Дата постановки на учет:").next.next.next
        date = date.get_text()
        return remove_bad_symbols(date)
    except AttributeError:
        # логии
        return None

In [199]:
# СТРАНА И КОД
def find_country_code_supplier(soup: object) -> tuple[str, str]:
    try:
        country_code = soup.get_text()
        country_code = remove_bad_symbols(country_code)

        contry_list = []
        code_list = []
        for i in country_code.split():
            if i.isdigit():
                code_list.append(i)
            else:
                contry_list.append(i)

        return " ".join(contry_list), "".join(code_list)
    except AttributeError:
        # логии
        return (None, None)

In [200]:
# АДРЕС МЕСТА НАХОЖДЕНИЯ
def find_adress_supplier(soup: object) -> str:
    try:
        adress = soup.get_text()
        adress = remove_bad_symbols(adress)

        return adress
    except AttributeError:
        # логии
        return None

In [201]:
# ПОЧТОВЫЙ АДРЕСС
def find_postal_adress_supplier(soup: object) -> str:
    try:
        adress = soup.get_text()
        adress = remove_bad_symbols(adress)

        return adress
    except AttributeError:
        # логии
        return None

In [202]:
# ТЕЛЕФОН, ЭЛЕКТРОННАЯ ПОЧТА
def find_contact_supplier(soup: object) -> str:
    try:
        contact = soup.get_text()
        contact = remove_bad_symbols(contact)

        return contact
    except AttributeError:
        # логии
        return None

In [203]:
# СТАТУС
def find_status_supplier(soup: object) -> str:
    try:
        status = soup.get_text()
        status = remove_bad_symbols(status)

        return status
    except AttributeError:
        # логии
        return None

In [204]:
def read_table_supplier(soup) -> tuple[list, list]:
    table_names = soup.find(
        "h2", class_="blockInfo__title", string="Информация о поставщиках"
    ).parent.find("tr", class_="tableBlock__row")

    table_values = soup.find(
        "h2", class_="blockInfo__title", string="Информация о поставщиках"
    ).parent.find_all("td", class_="tableBlock__col")

    if table_names is None or table_values is None:
        "логги"
        return None, None

    list_table_values = [element for element in table_values if type(element) is bs4.element.Tag]
    string = remove_bad_symbols(table_names.get_text())
    names_columns = []
    i_start = 0
    i_status = False
    for i in range(len(string)):
        if string[i].isupper() and not i_status:
            i_status = True
            i_start = i
        elif string[i].isupper() and i_status == True:
            names_columns.append(string[i_start:i])
            i_start = i
        elif i == len(string) - 1:
            names_columns.append(string[i_start : i + 1])
    return names_columns, list_table_values

In [205]:
def parsing_supplier(soup):
    """
    Парсинг данных поставщиков
    """
    names_columns, list_table_values = read_table_supplier(soup)

    if names_columns is None or list_table_values is None or len(list_table_values) == 0:
        print(1)
        return None

    for i_step, name_columns in enumerate(names_columns):
        if "Организация" in name_columns:
            full_name_supplier = find_full_name_supplier(list_table_values[i_step])
            inn_supplier = find_inn_supplier(list_table_values[i_step])
            kpp_supplier = find_kpp_supplier(list_table_values[i_step])
            code_okpo_supplier = find_code_okpo_supplier(list_table_values[i_step])
            date_registration_supplier = find_date_registration_supplier(
                list_table_values[i_step]
            )

            print("full_name_supplier", full_name_supplier)
            print("inn_supplier", inn_supplier)
            print("kpp_supplier", kpp_supplier)
            print("code_okpo_supplier", code_okpo_supplier)

            print("date_registration_supplier", date_registration_supplier)
            continue

        if "Страна, код" in name_columns:
            country, code = find_country_code_supplier(list_table_values[i_step])
            print("country", country)
            print("code", code)
            continue

        if "Адрес места нахождения" in name_columns:
            adress_supplier = find_adress_supplier(list_table_values[i_step])
            print("adress_supplier", adress_supplier)
            continue

        if "Почтовый адрес" in name_columns:
            postal_adress_supplier = find_postal_adress_supplier(list_table_values[i_step])
            print("postal_adress_supplier", postal_adress_supplier)
            continue

        if "Телефон, электронная почта" in name_columns:
            contact = find_contact_supplier(list_table_values[i_step])
            print("contact", contact)
            continue

        if "Статус" in name_columns:
            status_supplier = find_status_supplier(list_table_values[i_step])
            print("status_supplier", status_supplier)
            continue


parsing_supplier(soup)

full_name_supplier Виноградов Владимир Анатольевич
inn_supplier 690500008361
kpp_supplier None
code_okpo_supplier None
date_registration_supplier 21.02.2004
country Российская Федерация
code 643
adress_supplier г.Тверь,ул.Спартака, дом 43Б, кв 10
contact 7-4822-563042 vinogradov@mail.ru


## Платежи и объекты закупки 

In [206]:
url_payment_test = """https://zakupki.gov.ru/epz/contract/contractCard/
payment-info-and-target-of-order.html?reestrNumber="""

In [207]:
number_contract = "2780214400023000008"
url_info = f"{url_payment}{number_contract}"
res = requests.get(url_info, headers={"User-Agent": UserAgent().random})
soup = BeautifulSoup(res.text, "lxml")

In [208]:
# Объекты запупки
def find_contract_object(soup: object) -> str:
    try:
        obj = soup.find("div", class_="padBtm5 inline js-expand-all-list--not-count").get_text()
        return remove_bad_symbols(obj)
    except AttributeError:
        return None


find_contract_object(soup)

'1. крытый лыжный тоннель, абонемент для группы физ лиц 8-15 чел. (2 группы, 8 трен.)'

In [209]:
# КБК
def find_kbk(soup: object) -> str:
    try:
        code = soup.find("td", class_="tableBlock__col")
        code = remove_bad_symbols(code.get_text())

        return code.replace(" ", "")
    except AttributeError:
        # логии
        return None


find_status_supplier(soup)

'Карточка контрактаОфициальный сайт Единой информационной системы в сфере закупокМой регион: Личный кабинетЛичный кабинет44-Ф3Личный кабинет223-ФЗЛичный кабинет участника закупок Официальный сайт единой информационной системы в сфере закупок в информационно-телекоммуникационной сети Интернет (далее – Официальный сайт ЕИС) предназначен для обеспечения свободного и безвозмездного доступа к полной и достоверной информации о контрактной системе в сфере закупок и закупках товаров, работ, услуг, отдельными видами юридических лиц, а также для формирования, обработки и хранения такой информации. Порядок размещения информации на Официальном сайте ЕИС и ее содержание регламентируется Федеральным законом от 05.04.2013 № 44-ФЗ «О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд» и Федеральным законом от 18.07.2011 № 223-ФЗ «О закупках товаров, работ, услуг отдельными видами юридических лиц», а также соответствующими подзаконными актами.

In [210]:
number_contract = "0358200041314000090"
url_info = f"{url_payment}{number_contract}"
res = requests.get(url_info, headers={"User-Agent": UserAgent().random})
soup = BeautifulSoup(res.text, "lxml")

In [211]:
def get_page(url: str) -> BeautifulSoup:
    """
    Метод считывает заданную страницу,
    а потом преобразует в lxml
    """
    for num_attempt in range(10):
        res = requests.get(url, headers={"User-Agent": UserAgent().random})
        if res is not None and res.ok:
            return BeautifulSoup(res.text, "lxml")


number_contract = "1666105550722000041"
soup3 = get_page(f"{url_payment}{number_contract}#page-3")

In [212]:
def read_table_budget(soup: BeautifulSoup) -> list:
    soup = soup.find("table", id="contract_subjects")
    soup_table = soup.find_all("th", class_="tableBlock__col tableBlock__col_header")
    soup_table += soup.find_all(
        "th", class_="tableBlock__col tableBlock__col_right tableBlock__col_header"
    )

    list_result = []
    for col_name in soup_table:
        col_name = remove_bad_symbols(col_name.get_text())
        if col_name != "":
            list_result.append(col_name)

    return list_result


read_table_budget(soup)

['Наименование объекта закупки и его характеристики',
 'Позиции по КТРУ, ОКПД2',
 'Тип объекта закупки',
 'Количество товара, объем работы, услуги,Единица измерения',
 'Страна происхождения товара',
 'Цена за единицу измерения, ₽',
 'Сумма, ₽']

In [213]:
def write_info_object(number_contract: str):
    url_payment = "https://zakupki.gov.ru/epz/contract/contractCard/payment-info-and-target-of-order.html?reestrNumber="
    soup = get_page(f"{url_payment}{number_contract}")
    table_columns = read_table_budget(soup)
    print(table_columns)


number_contract = "1666105550722000041"
write_info_object(number_contract)

['Наименование объекта закупки и его характеристики', 'Позиции по КТРУ, ОКПД2', 'Тип объекта закупки', 'Количество товара, объем работы, услуги,Единица измерения', 'Страна происхождения товара', 'Цена за единицу измерения, ₽', 'Сумма, ₽']


In [214]:
def read_table_budget(soup: BeautifulSoup) -> list:
    soup_table = soup.find("table", id="contract_subjects")
    soup_table = soup_table.find("tbody", class_="tableBlock__body")
    list_values = soup_table.find_all("tr", class_="tableBlock__row")

    return list_values


for i in read_table_budget(soup)[1]:
    if remove_bad_symbols(i.get_text()) != "":
        print(remove_bad_symbols(i.get_text()))
    else:
        pass

In [215]:
def read_table_budget(soup) -> tuple[list, list]:
    table_names = soup.find(
        "h2", class_="blockInfo__title", string="Информация о поставщиках"
    ).parent.find("tr", class_="tableBlock__row")

    table_values = soup.find(
        "h2", class_="blockInfo__title", string="Информация о поставщиках"
    ).parent.find_all("td", class_="tableBlock__col")

    if table_names is None or table_values is None:
        "логги"
        return None, None

    list_table_values = [element for element in table_values if type(element) is bs4.element.Tag]
    string = remove_bad_symbols(table_names.get_text())
    names_columns = []
    i_start = 0
    i_status = False
    for i in range(len(string)):
        if string[i].isupper() and not i_status:
            i_status = True
            i_start = i
        elif string[i].isupper() and i_status == True:
            names_columns.append(string[i_start:i])
            i_start = i
        elif i == len(string) - 1:
            names_columns.append(string[i_start : i + 1])
    return names_columns, list_table_values

## Прасинг информации об организации

Добаить местонахождение

In [216]:
url_id = "https://zakupki.gov.ru/epz/organization/view/info.html?organizationCode="
code = "03553000005"
res = requests.get(f"{url_id}{code}", headers={"User-Agent": UserAgent().random})
soup = BeautifulSoup(res.text, "html")
res.ok

True

In [217]:
# Доступ организации заблокирован организацией, оказывающей услуги по обслуживанию пользователей ЕИС
def find_access_blocking(soup: BeautifulSoup) -> str:
    try:
        query_1 = "Доступ организации заблокирован организацией,"
        query_2 = "оказывающей услуги по обслуживанию пользователей ЕИС"

        query = query_1 + " " + query_2
        soup = soup.find(
            "span",
            class_="section__title",
            string=query,
        )
        access = soup.parent.find("span", class_="section__info")
        return remove_bad_symbols(access.get_text())
    except AttributeError:
        # логии
        return None


find_access_blocking(soup)

'Да'

In [220]:
# Местонахождение
def find_adress(soup: BeautifulSoup) -> str:
    try:
        name = soup.find("span", class_="section__title", string="Место нахождения")
        name = name.parent.find("span", class_="section__info")
        name = remove_bad_symbols(name.get_text())
        return remove_bad_symbols(name)
    except AttributeError:
        # логии
        return None

find_adress(soup)

'Российская Федерация, 442600, Пензенская обл, Спасский р-н, Спасск г, КОММУНАЛЬНАЯ, 39'

In [70]:
# ПОЛНОЕ НАИМЕНОВАНИЕ
def find_full_name(soup: BeautifulSoup) -> str:
    try:
        name = soup.find("span", class_="section__title", string="Полное наименование")
        name = name.parent.find("span", class_="section__info")
        name = remove_bad_symbols(name.get_text())
        return name
    except AttributeError:
        # логии
        return None


find_full_name(soup)

'ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИЯ "СПАССКАЯ РАЙОННАЯ БОЛЬНИЦА"'

In [71]:
# СОКРАЩЕННОЕ НАИМЕНОВАНИЕ
def find_short_name(soup: BeautifulSoup) -> str:
    try:
        name = soup.find("span", class_="section__title", string="Сокращенное наименование")
        name = name.parent.find("span", class_="section__info")
        name = remove_bad_symbols(name.get_text())
        return name
    except AttributeError:
        # логии
        return None


find_short_name(soup)

'ГБУЗ "Спасская РБ"'

In [72]:
# КОД ПО СВОДНОМУ РЕГИСТРУ
def find_code_registr(soup: BeautifulSoup) -> str:
    try:
        code = soup.find("span", class_="section__title", string="Код по Сводному реестру")
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_code_registr(soup)

'01552000016'

In [73]:
# Дата регистрации
def find_date_registration(soup: BeautifulSoup) -> str:
    try:
        code = soup.find("span", class_="section__title", string="Дата регистрации")
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_date_registration(soup)

'31.12.2010'

In [74]:
# Дата/время последнего изменения записи об организации
def find_date_last_change(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="Дата/время последнего изменения сведений об организации",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_date_last_change(soup)

In [75]:
# ИНН
def find_inn(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="ИНН",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_inn(soup)

'5807001820'

In [76]:
# КПП
def find_kpp(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="КПП",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_kpp(soup)

'580701001'

In [77]:
# ОГРН
def find_ogrn(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="ОГРН",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_ogrn(soup)

'1035801600713'

In [78]:
# ОКТМО
def find_oktmo(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="ОКТМО",
        )
        if code is None:
            code = soup.find(
                "span",
                class_="section__title",
                string="Код ОКТМО",
            )

        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_oktmo(soup)

'56000000'

In [79]:
# МЕСТО НАХОЖДЕНИЯ
def find_location(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="Место нахождения",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_location(soup)

'Российская Федерация, 442600, Пензенская обл, Спасский р-н, Спасск г, КОММУНАЛЬНАЯ, 39'

In [80]:
# ИКУ
def find_iky(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="ИКУ",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_location(soup)

'Российская Федерация, 442600, Пензенская обл, Спасский р-н, Спасск г, КОММУНАЛЬНАЯ, 39'

In [81]:
# ДАТА ПРИСВОЕНИЯ ИКУ
def find_date_iky(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="Дата присвоения ИКУ",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_date_iky(soup)

'01.01.2015'

In [82]:
# КОД ПО ОФКС
def find_code_okfs(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="Код по ОКФС",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_code_okfs(soup)

In [83]:
# НАИМЕНОВАНИЕ СОБСТВЕННОСТИ
def find_name_property(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="Наименование",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_name_property(soup)

'Пензенская область'

In [84]:
# КОД ПО ОКОПФ
def find_code_okpf(soup: BeautifulSoup) -> str:
    try:
        code = soup.find_all(
            "span",
            class_="section__title",
            string="Код по ОКОПФ",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_code_okpf(soup)

In [85]:
# КОД ПО ОКОПФ
def find_okopf_name(soup: BeautifulSoup) -> str:
    try:
        code = soup.find_all(
            "span",
            class_="section__title",
            string="Наименование",
        )[1]
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_okopf_name(soup)

'МИНИСТЕРСТВО ЗДРАВООХРАНЕНИЯ ПЕНЗЕНСКОЙ ОБЛАСТИ'

## Парсинг другого

In [86]:
code = "731554"
res = requests.get(
    f"https://zakupki.gov.ru/epz/organization/view/info.html?organizationId={code}&tab=other",
    headers={"User-Agent": UserAgent().random},
)
soup = BeautifulSoup(res.text, "html")
res.ok

True

In [87]:
# ПОЛНОМОЧИЯ ОРГАНИЗАЦИИ
def find_credentials(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="Полномочия организации",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_credentials(soup)

'Заказчик (с 26.07.2017 )'

In [88]:
# ДАТА ПОСТАНОВКИ НА УЧЕТ В НАЛОГОВОМ ОРГАНЕ
def find_date_registration_tax(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="Дата постановки организации на учет в налоговом органе",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_date_registration_tax(soup)

'28.03.2022'

In [89]:
# ТИП ОРГАНИЗАЦИИ
def find_organization_type(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="Тип организации",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_organization_type(soup)

'Казенное учреждение'

In [90]:
# УРОВЕНЬ ОРГАНИЗАЦИИ
def find_organization_level(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="Уровень организации",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_organization_level(soup)

'Муниципальный уровень'

In [91]:
# ОКПО
def find_okpo_code(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="ОКПО",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_okpo_code(soup)

'51666568'

In [92]:
# ОКВЭД
def find_okfd_code(soup: BeautifulSoup) -> str:
    try:
        code = soup.find(
            "span",
            class_="section__title",
            string="ОКВЭД",
        )
        code = code.parent.find("span", class_="section__info")
        code = remove_bad_symbols(code.get_text())
        return code
    except AttributeError:
        # логии
        return None


find_okfd_code(soup)

'85.11: Образование дошкольное'

## Данные по Бюджету

In [93]:
# ТАБЛИЦА БЮДЖЕТА
def find_budget_table(soup: BeautifulSoup) -> str:
    try:
        table = soup.find(
            "h2",
            class_="blockInfo__title",
            string="Бюджеты",
        )
        table = table.parent.find("tr", class_="tableBlock__row")
        return remove_bad_symbols(table.get_text())
    except AttributeError:
        # логии
        return None

In [94]:
# ДАННЫЕ ПО БЮДЖЕТУ
def find_budget_data(soup: BeautifulSoup) -> str:
    try:
        data = soup.find(
            "h2",
            class_="blockInfo__title",
            string="Бюджеты",
        )
        data = data.parent.find_all("tr", class_="tableBlock__row")[1].find_all("td")
        return data
    except AttributeError:
        # логии
        return None

In [95]:
def parsing_budget_table(soup):
    string = find_budget_table(soup)
    if string is None:
        return None, None
    stri = []
    i_start = 0
    i_status = False
    for i in range(len(string)):
        if string[i].isupper() and i_status == False:
            i_status = True
            i_start = i
        elif string[i].isupper() and i_status == True:
            stri.append(string[i_start : i - 1])
            i_start = i
        elif i == len(string) - 1:
            stri.append(string[i_start : i + 1])

    budget_data = find_budget_data(soup)
    if budget_data is None:
        return None, None

    code_budget = None
    name_budget = None
    for i_step, i in enumerate(stri):
        i_step += 1
        if "Код" in i:
            code_budget = remove_bad_symbols(budget_data[i_step].get_text())
        if "Наименование" in i:
            name_budget = remove_bad_symbols(budget_data[i_step].get_text())
    return code_budget, name_budget


parsing_budget_table(soup)

('25030091',
 'Бюджет муниципального образования "Камызякский муниципальный район Астраханской области"')

## Контактные данные организации

In [96]:
# КОНТАКТНАЯ ИНФОРМАЦИЯ
def find_contact_info(soup: BeautifulSoup) -> str:
    try:
        info = soup.find("h2", class_="blockInfo__title", string="Контактная информация").parent
        return info
    except AttributeError:
        # логии
        return None


contact_info = find_contact_info(soup)

In [97]:
# ТЕЛЕФОН
def find_telephone(soup: BeautifulSoup) -> str:
    try:
        soup = soup.find("span", class_="section__title", string="Телефон")
        telephone = soup.parent.find("span", class_="section__info")
        return remove_bad_symbols(telephone.get_text())
    except AttributeError:
        # логии
        return None


find_telephone(contact_info)

'+7 (85145) 91249'

In [98]:
# ФАКС
def find_fax(soup: BeautifulSoup) -> str:
    try:
        soup = soup.find("span", class_="section__title", string="Факс")
        telephone = soup.parent.find("span", class_="section__info")
        return remove_bad_symbols(telephone.get_text())
    except AttributeError:
        # логии
        return None


find_fax(contact_info)

'+7 (85145) 91245'

In [99]:
# ПОЧТОВЫЙ АДРЕС
def find_postal_adress(soup: BeautifulSoup) -> str:
    try:
        soup = soup.find("span", class_="section__title", string="Почтовый адрес")
        telephone = soup.parent.find("span", class_="section__info")
        return remove_bad_symbols(telephone.get_text())
    except AttributeError:
        # логии
        return None


find_postal_adress(contact_info)

'Российская Федерация, 416340, Астраханская обл, Камызякский р-н, Камызяк г, УЛ. ТАРАКАНОВА, Д.11'

In [100]:
# КОНТАКТНЫЙ АДРЕС ЭЛЕКТРОННОЙ ПОЧТЫ
def find_email(soup: BeautifulSoup) -> str:
    try:
        soup = soup.find(
            "span", class_="section__title", string="Контактный адрес электронной почты"
        )
        telephone = soup.parent.find("span", class_="section__info")
        return remove_bad_symbols(telephone.get_text())
    except AttributeError:
        # логии
        return None


find_email(contact_info)

'kam_ds1@mail.ru'

In [101]:
# САЙТ
def find_site(soup: BeautifulSoup) -> str:
    try:
        soup = soup.find(
            "span", class_="section__title", string="Адрес организации в сети Интернет"
        )
        telephone = soup.parent.find("span", class_="section__info")
        return remove_bad_symbols(telephone.get_text())
    except AttributeError:
        # логии
        return None


find_site(contact_info)

'http://'

In [102]:
# КОНТАКТНОЕ ЛИЦО
def find_contact_person(soup: BeautifulSoup) -> str:
    try:
        soup = soup.find("span", class_="section__title", string="Контактное лицо")
        telephone = soup.parent.find("span", class_="section__info")
        return remove_bad_symbols(telephone.get_text())
    except AttributeError:
        # логии
        return None


find_contact_person(contact_info)

'Барышева Л. Н.'

In [103]:
# ЧАСОВАЯ ЗОНА
def find_time_zone(soup: BeautifulSoup) -> str:
    try:
        soup = soup.find("span", class_="section__title", string="Часовая зона")
        telephone = soup.parent.find("span", class_="section__info")
        return remove_bad_symbols(telephone.get_text())
    except AttributeError:
        # логии
        return None


find_time_zone(contact_info)

'MSK+1 (UTC+4) Самара'

## Парсинг особых орагнизаций

In [210]:
url_id = "https://zakupki.gov.ru/epz/organization/view/info.html?organizationCode="
code = "03553000005"
res = requests.get(f"{url_id}{code}", headers={"User-Agent": UserAgent().random})
soup = BeautifulSoup(res.text, "html")
res.ok

True

Скорее всего здесь не найти: полномочия организации

In [211]:
def find_access_blocking(soup: BeautifulSoup) -> str:
    try:
        soup = soup.find(
            "span",
            class_="section__title",
            string="Доступ организации заблокирован организацией, оказывающей услуги по обслуживанию пользователей ЕИС",
        )
        access = soup.parent.find("span", class_="section__info")
        return remove_bad_symbols(access.get_text())
    except AttributeError:
        # логии
        return None


find_access_blocking(soup)

'Да'

In [212]:
# ПОЛНОМОЧИЯ ОРГАНИЗАЦИИ
find_credentials(soup)

In [213]:
# ДАТА ПОСТАНОВКИ НА УЧЕТ В НАЛОГОВОМ ОРГАНЕ
find_date_registration_tax(soup)

'11.04.1994'

In [214]:
find_organization_type(soup)

In [215]:
find_organization_level(soup)

In [216]:
find_okpo_code(soup)

'01938731'

In [217]:
find_okfd_code(soup)

'85.11.1: Деятельность больничных учреждений широкого профиля и специализированных 85.12: Образование начальное общее 85.13: Образование основное общее 85.14.3: Деятельность медицинских лабораторий'

In [218]:
parsing_budget_table(soup)

('55020001', 'Бюджет Пензенской области')

In [219]:
contact_info = find_contact_info(soup)

In [221]:
find_telephone(contact_info)

'+7 (84151) 31645'

In [222]:
find_fax(contact_info)

'+7 (84151) 31645'

In [223]:
find_postal_adress(contact_info)

'Российская Федерация, 442600, Пензенская обл, Спасский р-н, Спасск г, КОММУНАЛЬНАЯ, 39'

In [224]:
find_email(contact_info)

In [225]:
find_site(contact_info)

In [226]:
find_contact_person(contact_info)

'Зайцева Е. А.'

In [227]:
find_time_zone(contact_info)

'MSK (UTC+3) Москва'